In [1]:
import numpy as np
import pandas as pd
import os
import sqlalchemy as sqla
import sqlite3 as sql
from sqlite3 import Error
import re

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
file_loc = os.getcwd()

In [3]:
db= sql.connect(file_loc+"\data_pred.db")
df=pd.read_sql("SELECT * FROM loan",db)

In [4]:
df.shape

(38771, 65)

In [21]:
data = df.copy()

col_drop = ['grade','sub_grade','issue_d','earliest_cr_line','zip_code','collection_recovery_fee','avg_cur_bal',
            'total_pymnt', 'total_pymnt_inv','last_credit_pull_d', 'funded_amnt','funded_amnt_inv','out_prncp_inv']
data = data.drop(col_drop, axis = 1)
data = data.reset_index(drop=True)
data.shape

(38771, 52)

In [22]:
y_temp = data['loan_status']

In [23]:
# recode loan_status
y = y_temp.copy()
for i in range(len(y)):
    if (y_temp[i] == 'Fully Paid') or (y_temp[i] == 'Current'):
        y[i] = 'good'
    else:
        y[i] = 'bad'

In [24]:
y.head()

0    good
1    good
2    good
3    good
4    good
Name: loan_status, dtype: object

In [25]:
purpose_temp = data['purpose']

In [26]:
#recode purposes
purpose = purpose_temp.copy()
for i in range(len(purpose)):
    if (purpose_temp[i] == 'credit_card') or ( purpose_temp[i] == 'debt_consolidation') or (purpose_temp[i] == 'small_business') or (purpose_temp[i] == 'vacation') or (purpose_temp[i] == 'other'):
        purpose[i] = 'financial'
    elif purpose_temp[i] == 'medical':
        purpose[i] == 'medical'
    else:
        purpose[i] = 'property'

In [27]:
purpose.head()

0    financial
1    financial
2    financial
3    financial
4    financial
Name: purpose, dtype: object

In [28]:
addr_state_temp = data['addr_state']
#recode addr_state
addr_state = addr_state_temp.copy()
for i in range(len(y)):
    if addr_state_temp[i] == 'CA':
        addr_state[i] = 'CA'
    else:
        addr_state[i] = 'other'
        
addr_state.head()

0    other
1    other
2    other
3    other
4    other
Name: addr_state, dtype: object

In [29]:
verification_status_temp = data['verification_status']

In [30]:
#recode addr_state
verification_status = verification_status_temp.copy()
for i in range(len(y)):
    if verification_status_temp[i] == 'Not Verified':
        verification_status[i] = 'not verified'
    else:
        verification_status[i] = 'verified'

In [31]:
verification_status.head()

0    not verified
1        verified
2    not verified
3    not verified
4    not verified
Name: verification_status, dtype: object

In [32]:
#Create dummy variables
dummy1 = pd.get_dummies(y)
dummy2 = pd.get_dummies(purpose)
dummy3 = pd.get_dummies(addr_state)
dummy4 = pd.get_dummies(data['term'])
dummy5 = pd.get_dummies(data['home_ownership'])
dummy6 = pd.get_dummies(verification_status)
dummy7 = pd.get_dummies(data['initial_list_status'])
dummy8 = pd.get_dummies(data['application_type'])

In [33]:
#Insert dummy variables into dataset
data['loan_status'] = dummy1['good']
data['addr_state'] = dummy3['CA']
data['term'] = dummy4[' 36 months']
data['verification_status'] = dummy6['verified']
data['initial_list_status'] = dummy7['w']
data['application_type'] = dummy8['Individual']

data = data.drop(['purpose','home_ownership'], axis = 1)
data = pd.concat([data, dummy2.iloc[:,[0,2]],dummy5.iloc[:,1:4]],axis = 1)

# convert all data as float
data['int_rate'] = [x[:-1] for x in data['int_rate']]
data = data.astype(float)
data['int_rate'] = data['int_rate']/100

data.head()

loan_amnt  term  int_rate  installment  annual_inc  verification_status  \
0     3000.0   1.0    0.1041        97.39     60000.0                  0.0   
1    14100.0   1.0    0.2387       552.23    128000.0                  1.0   
2    10000.0   1.0    0.0607       304.54     50000.0                  0.0   
3    40000.0   0.0    0.0943       838.71    150000.0                  0.0   
4     6300.0   1.0    0.0943       201.61     60000.0                  0.0   

   loan_status  addr_state  delinq_2yrs  open_acc  ...   tax_liens  \
0          1.0         0.0          0.0      20.0  ...         0.0   
1          1.0         0.0          1.0      12.0  ...         0.0   
2          1.0         0.0          0.0       9.0  ...         0.0   
3          1.0         0.0          0.0       6.0  ...         0.0   
4          1.0         0.0          0.0       7.0  ...         1.0   

   tot_hi_cred_lim  total_bal_ex_mort  total_bc_limit  \
0         248466.0            56265.0         56000.0   
1         326745.0           126106.0         17800.0   
2          33788.0            12083.0         13300.0   
3          74455.0            15615.0         44200.0   
4          63103.0            54565.0          9000.0   

   total_il_high_credit_limit  financial  property  MORTGAGE  OWN  RENT  
0                     47716.0        1.0       0.0       1.0  0.0   0.0  
1                    101375.0        1.0       0.0       0.0  1.0   0.0  
2                     11088.0        1.0       0.0       0.0  1.0   0.0  
3                     27755.0        1.0       0.0       1.0  0.0   0.0  
4                     51403.0        1.0       0.0       0.0  0.0   1.0  

[5 rows x 55 columns]

In [34]:
# store the categorical variable indexes
data_category = ['loan_status','verification_status','addr_state', 'term', 'initial_list_status',
                 'application_type','financial','property','MORTGAGE','OWN','RENT']

In [35]:
from  sklearn.preprocessing  import scale

data_lasso = data.copy()

# Scale the numerical variables
data_lasso[data.columns[~data_lasso.columns.isin(data_category)]]=scale(data_lasso[data.columns[~data_lasso.columns.isin(data_category)]])

In [36]:
X = data_lasso[data_lasso.columns.difference(['loan_status'])].values
y = data_lasso.loc[:, 'loan_status'].values
X=X.astype(float)

In [38]:
selected_X = ['loan_amnt', 'term', 'int_rate', 'annual_inc', 'out_prncp',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'last_pymnt_amnt', 'num_bc_tl', 'num_op_rev_tl', 'num_rev_tl_bal_gt_0',
       'num_tl_op_past_12m', 'total_bc_limit', 'financial', 'property',
       'MORTGAGE']
data_fnl = pd.concat([data_lasso[selected_X], data_lasso['loan_status']],axis = 1)

In [39]:
data_fnl.shape

(38771, 19)

In [41]:
db= sql.connect(file_loc+"\data_pred.db")
data_fnl.to_sql("pred",db,index=False)